In [624]:
import pandas as pd
import numpy as np

In [625]:
df = pd.read_csv("../disease_symptoms_weight.csv",index_col="Unnamed: 0")

In [626]:
dis_df=pd.read_csv('../data.csv',index_col='disease')
dis_df.head()

,description,precautions
disease,,
(vertigo) paroymsal positional vertigo,Benign paroxysmal positional vertigo (BPPV) is...,"lie down,avoid sudden change in body,avoid abr..."
acne,"Acne vulgaris is the formation of comedones, p...","bath twice,avoid fatty spicy food,drink plenty..."
aids,Acquired immunodeficiency syndrome (AIDS) is a...,"avoid open cuts,wear ppe if possible,consult d..."
alcoholic hepatitis,"Alcoholic hepatitis is a diseased, inflammator...","stop alcohol consumption,consult doctor,medica..."
allergy,An allergy is an immune system response to a f...,"apply calamine,cover area with bandage,use ice..."


In [627]:
df.head()

,abdominal_pain,abnormal_menstruation,acidity,acute_liver_failure,altered_sensorium,anxiety,back_pain,belly_pain,blackheads,bladder_discomfort,...,weakness_in_limbs,weakness_of_one_body_side,weight_gain,weight_loss,yellow_crust_ooze,yellow_urine,yellowing_of_eyes,yellowish_skin,Disease,normalised_mean
1186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,(vertigo) paroymsal positional vertigo,4.6
2865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,(vertigo) paroymsal positional vertigo,4.5
1187,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,(vertigo) paroymsal positional vertigo,4.6
4136,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,(vertigo) paroymsal positional vertigo,4.5
1188,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,(vertigo) paroymsal positional vertigo,4.5


In [628]:
data = df.iloc[:,:-2]
label = df['Disease']

In [629]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,f1_score

In [630]:
x_train,x_test,y_train,y_test = train_test_split(data,label,test_size=0.3,shuffle=True)

In [631]:
symp_weight_df = pd.read_csv('../symptom_severity.csv')
symp_weight_df["Symptom"].drop_duplicates(inplace=True)
symp_weight_df.set_index("Symptom", inplace=True)
symp_weight_df.sort_index(inplace=True)
symp_weight_df

,weight
Symptom,
abdominal_pain,4
abnormal_menstruation,6
acidity,3
acute_liver_failure,6
altered_sensorium,2
...,...
weight_loss,3
yellow_crust_ooze,3
yellow_urine,4


In [632]:
sympts=df.iloc[:,:-2].columns.tolist()

In [633]:
def weightadder(in_lst):

    input_symptoms = in_lst

    input_symptoms_list = []

    for s in sympts:
        if s in input_symptoms:
            weight = symp_weight_df.loc[s, "weight"]
            input_symptoms_list.append(weight)
        else:
            input_symptoms_list.append(0)

    return input_symptoms_list


In [634]:
# to_pre = weightadder(["unsteadiness", "spinning_movements", "headache"])
# to_pre = weightadder(["spinning_movements", "headache"])
to_pre = weightadder(["runny_nose","loss_of_smell","mild_fever"])

print(to_pre)
# to_pre = pd.DataFrame(to_pre).transpose()
to_pre = pd.DataFrame(to_pre)
sy = to_pre.loc[to_pre[0]!=0]
to_pre = to_pre.transpose()
to_pre.columns = sympts

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### RandomForest

In [635]:
from sklearn.ensemble import RandomForestClassifier

In [636]:
m1 = RandomForestClassifier()

In [637]:
m1.fit(x_train,y_train)

RandomForestClassifier()

In [638]:
preds = m1.predict(x_test)

In [639]:
f1 = f1_score(y_test, preds, average='macro') * 100
acu = accuracy_score(y_test, preds)*100
pre = precision_score(y_test, preds, average="weighted")
print('F1-score% = ', f1)
print('Accuracy% = ', acu)
print('Precision = ', pre)


F1-score% =  100.0
Accuracy% =  100.0
Precision =  1.0


In [640]:
p_proba = m1.predict_proba(to_pre)
p_proba

array([[0.01, 0.04, 0.04, 0.  , 0.01, 0.02, 0.02, 0.01, 0.07, 0.  , 0.22,
        0.  , 0.01, 0.03, 0.  , 0.01, 0.03, 0.  , 0.01, 0.14, 0.  , 0.  ,
        0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.02, 0.  , 0.  , 0.02, 0.01,
        0.05, 0.  , 0.  , 0.01, 0.04, 0.  , 0.15, 0.02]])

In [641]:
top_n_category_predictions = np.argsort(p_proba)[:, :-41:-1]
order = list(top_n_category_predictions[0])[:10]

In [642]:
dis_series = pd.Series(dis_df.index)
order = dis_series.loc[order].values.tolist()
order

['common cold',
 'urinary tract infection',
 'hepatitis a',
 'chicken pox',
 'paralysis (brain hemorrhage)',
 'acne',
 'tuberculosis',
 'aids',
 'gastroenteritis',
 'dimorphic hemorrhoids(piles)']

### DecisioTree

In [643]:
from sklearn.tree import DecisionTreeClassifier 

In [644]:
m1 = DecisionTreeClassifier()

In [645]:
m1.fit(x_train,y_train)

DecisionTreeClassifier()

In [646]:
preds = m1.predict(x_test)

In [647]:
f1 = f1_score(y_test, preds, average='macro') * 100
acu = accuracy_score(y_test, preds)*100
pre = precision_score(y_test, preds, average="weighted")
print('F1-score% = ', f1)
print('Accuracy% = ', acu)
print('Precision = ', pre)


F1-score% =  100.0
Accuracy% =  100.0
Precision =  1.0


In [648]:
p_proba = m1.predict_proba(to_pre)
p_proba

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [649]:
top_n_category_predictions = np.argsort(p_proba)[:, :-41:-1]
order = list(top_n_category_predictions[0])[:10]

In [650]:
dis_series = pd.Series(dis_df.index)
order = dis_series.loc[order].values.tolist()
order

['osteoarthristis',
 'varicose veins',
 'chronic cholestasis',
 'gerd',
 'gastroenteritis',
 'fungal infection',
 'drug reaction',
 'dimorphic hemorrhoids(piles)',
 'diabetes',
 'dengue']

### SVM

In [651]:
from sklearn.svm import SVC 

In [652]:
m1 = SVC(probability=True)

In [653]:
m1.fit(x_train,y_train)

SVC(probability=True)

In [654]:
preds = m1.predict(x_test)

In [655]:
f1 = f1_score(y_test, preds, average='macro') * 100
acu = accuracy_score(y_test, preds)*100
pre = precision_score(y_test, preds, average="weighted")
print('F1-score% = ', f1)
print('Accuracy% = ', acu)
print('Precision = ', pre)


F1-score% =  100.0
Accuracy% =  100.0
Precision =  1.0


In [656]:
p_proba = m1.predict_proba(to_pre)
p_proba

array([[0.01521123, 0.12937087, 0.02690811, 0.0111892 , 0.04621299,
        0.03658165, 0.01629538, 0.02443385, 0.01344678, 0.01832269,
        0.00615343, 0.00673938, 0.00841003, 0.01546678, 0.02563748,
        0.07482646, 0.03748831, 0.01587903, 0.03876218, 0.0137641 ,
        0.00676162, 0.01891794, 0.00936875, 0.00523814, 0.03105461,
        0.00837494, 0.00717209, 0.00570398, 0.04212105, 0.01476178,
        0.01307159, 0.01380483, 0.03488491, 0.06541563, 0.01529056,
        0.00608449, 0.05787561, 0.00517488, 0.00596883, 0.04018316,
        0.01167063]])

In [657]:
top_n_category_predictions = np.argsort(p_proba)[:, :-41:-1]
order = list(top_n_category_predictions[0])[:10]

In [658]:
dis_series = pd.Series(dis_df.index)
order = dis_series.loc[order].values.tolist()
order

['acne',
 'fungal infection',
 'paralysis (brain hemorrhage)',
 'psoriasis',
 'allergy',
 'impetigo',
 'urinary tract infection',
 'heart attack',
 'gastroenteritis',
 'arthritis']